In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

/Users/amlannag/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Loads the data using tensorflow library 
fw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

In [3]:
#The images from the data set
X = lfw_people.images
#The labels from the data set
Y = lfw_people.target
#Prints the max and mean to see if it has been normalised in greyscale 
print("X_min:",X.min(),"X_train_max:", X.max())
#Splits into testing and training
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
#TF and numpy require 4d arrays, this new axis represents the number of chanels 
#in the data which is 1 as we are using grey scale 
X_train = X_train[:, :, :,np.newaxis]
X_test = X_test[:, :, :,np.newaxis]
print("X_train shape:", X_train.shape)

X_min: 0.0 X_train_max: 1.0
X_train shape: (901, 50, 37, 1)


In [4]:
#Records the number of samples and height/weidth of the images
n_samples, h, w = lfw_people.images.shape
#Records the number of classes in teh y_train
n_classes = len(np.unique(y_train))
#The batch size used
batch_size = 80
#The number of categories we have 
categories = n_classes

In [5]:
#Takes in an image of height h and width w
inputs = Input(shape=(h,w,1))
#Creates 32 convolutions/feature maps 
net1 = Conv2D(filters = 32, kernel_size = (3,3), padding="same", activation="relu")(inputs)
#Down sampling preserves the important information
pool1 = MaxPooling2D(pool_size=(2,2))(net1)
#2nd convolutional layer
net2 = Conv2D(filters = 32, kernel_size = (3,3), padding="same", activation="relu")(pool1)
#2nd max pooling layer
pool2 = MaxPooling2D(pool_size=(2,2))(net2)
#Flattens the 2d data into 1D vector so it can be fed to the fully connected layer 
flat = Flatten()(pool2)
#Find the optimal information from the convoutions to output the 
net3 = Dense(128, activation="relu")(flat)
#Turns value from 0-1 using sigmoid function. 
output = Dense(categories, activation="softmax")(net3)

In [6]:
#Changing the output to probabilities using onehot encoding
y_train_cat = to_categorical(y_train, categories)
y_test_cat = to_categorical(y_test, categories)

In [7]:
#Defines the model
model = Model(inputs, output)
#Creating the optimise
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

#Runs training and does back proagation
model.fit(X_train, y_train_cat,
                epochs=15,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test, y_test_cat))

Epoch 1/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.3757 - loss: 1.7606 - val_accuracy: 0.4289 - val_loss: 1.6842
Epoch 2/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.4036 - loss: 1.6955 - val_accuracy: 0.4289 - val_loss: 1.6229
Epoch 3/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.4219 - loss: 1.6328 - val_accuracy: 0.4289 - val_loss: 1.5762
Epoch 4/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.4172 - loss: 1.5921 - val_accuracy: 0.4341 - val_loss: 1.4910
Epoch 5/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.4606 - loss: 1.4746 - val_accuracy: 0.5504 - val_loss: 1.3978
Epoch 6/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.5459 - loss: 1.3268 - val_accuracy: 0.5401 - val_loss: 1.2411
Epoch 7/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 0.5839 - loss: 1.1935 - val_accuracy: 0.6512 - val_loss: 1.1356
Epoch 8/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.6568 - loss: 1.0482 - val_accuracy: 0.6460 - 

In [8]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)
print(predictions)

#print performance to compare with previous RF versions
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[3 3 6 3 3 3 4 1 3 3 3 3 3 4 3 3 3 3 3 4 1 0 3 0 0 1 0 3 3 3 2 3 3 3 3 3 3
 3 3 1 3 1 3 1 1 1 4 3 0 3 3 3 0 3 6 2 1 3 5 3 1 1 0 4 0 5 6 4 1 3 6 6 3 3
 3 2 1 6 6 6 3 0 4 3 3 3 3 3 3 3 3 6 3 4 1 6 1 1 6 6 3 1 3 1 3 3 3 3 3 1 4
 1 4 4 3 1 3 4 1 3 1 3 3 0 3 4 4 6 1 3 6 6 6 3 4 4 3 3 0 6 2 2 5 1 3 5 1 3
 6 1 1 1 1 3 3 3 6 0 1 0 6 5 5 1 3 1 5 1 0 3 1 1 6 1 5 6 3 2 2 4 3 3 3 6 3
 3 3 3 3 2 3 2 3 2 6 3 3 6 3 6 3 2 1 2 3 2 6 2 0 6 1 5 3 3 3 3 3 0 0 1 2 3
 1 1 6 3 3 3 1 3 3 3 1 0 3 1 6 6 6 3 3 1 6 4 3 0 3 3 3 4 4 3 3 0 3 4 3 1 6
 0 3 3 6 1 3 6 6 1 6 6 1 1 3 3 6 3 3 3 3 3 1 2 3 3 1 0 3 3 3 4 3 3 5 1 3 1
 4 5 3 6 3 6 6 2 1 3 1 3 1 3 3 3 0 6 3 3 6 1 3 1 5 6 0 1 2 5 4 6 3 6 6 5 2
 3 4 1 6 4 3 3 2 3 6 3 1 6 1 3 0 1 6 0 6 5 3 1 3 1 6 3 3 2 3 0 1 0 3 6 3 3
 3 3 3 6 4 3 0 0 1 3 6 4 3 3 6 6 0]
              precision    recall  f1-score   support

           0       0.48      0.83      0.61        18
           1       0.88      0.88      0.88        69
           2